In [1]:
# EEE-INST-002: Instructions for EEE Parts Selection, Screening, Qualification,and Derating
import math


def fixed_film_chip_resistor(value, applied_V, ambient_T, nominal_P, nominal_V):
    """
    Resistance Derating for Fixed Film Chip Resistor, style RM

    value = resistance value in ohms
    applied_V = applied voltage in volts
    ambient_T = ambient temperature in degrees C
    nominal_P = nominal power in watts (e.g. 0.1 for 0603)
    nominal_V = nominal voltage in volts (e.g. 50 for 0603)
    """
    return _resistor_derating(value, applied_V, ambient_T, nominal_P, nominal_V, 0.6, 0.8, 70, 118)


def _resistor_derating(value, applied_V, ambient_T, nominal_P, nominal_V, derating_factor_P, derating_factor_V, derating_T1, derating_T2):
    """
    Resistance Derating for Fixed Film Chip Resistor, style RM

    value = resistance value in ohms
    applied_V = applied voltage in volts
    ambient_T = ambient temperature in degrees C
    nominal_P = nominal power in watts (e.g. 0.1 for 0603)
    nominal_V = nominal voltage in volts (e.g. 50 for 0603)
    derating_factor_P = derating factor for power (see table 4) (e.g. 0.6 for RM)
    derating_factor_V = derating factor for voltage (see table 4) (e.g. 0.8 for RM)
    derating_T1 = first temperature in degrees C (e.g. 70 for RM)
    derating_T2 = second temperature in degrees C (e.g. 118 for RM)
    """
    T1 = derating_T1
    T2 = derating_T2

    if ambient_T <= T1:
        Pmax = derating_factor_P * nominal_P
    elif ambient_T <= T2:
        # y = ax + b
        # a = (y2-y1)/(x2-x1) = (-0.6)/(T2-T1) = 0
        # b = y1 = 0.6
        Pmax = ((-derating_factor_P/(T2-T1)) *
                          (ambient_T-T1) + derating_factor_P) * nominal_P
    else:
        print("Ambient temperature exceeds power derating: " + str(ambient_T) + "°C > " + str(T2) + "°C")
        return False
            
    applied_P = (applied_V**2)/value
    if applied_P > Pmax:
        print("Applied power exceeds power derating: " + str(round(applied_P,3)) + "W > " + str(round(Pmax,3))+"W")
        print("Maximum allowed voltage: " + str(round(math.sqrt(Pmax*value),1)) + "V")
        return False
    
    Vmax = min(math.sqrt(Pmax*value), derating_factor_V * nominal_V)

    if applied_V > Vmax:
        print("Applied voltage exceeds voltage derating: " + str(applied_V) + " > " + str(Vmax))
        return False

    return True

In [2]:
# Example
nominal_P_0603 = 0.1 # watts
nominal_V_0603 = 50 # volts

nominal_P_1206 = 0.25 # watts
current_sense_P_1206 = 0.5 # watts (0.25W and 1W exist as well)
nominal_V_1206 = 200 # volts

nominal_P_2512 = 1 # watts (normal 2512 is only 1W)
nominal_V_2512 = 200 # volts


# Pick a resistor value
resistorValue = 180 # ohms


# Pick a resistor package
nominal_P = nominal_P_0603
nominal_V = nominal_V_0603

# Set the conditions : ambient temperature, applied voltage
ambient_T = 50 # degrees C
apply_V = 3 # volts

print(fixed_film_chip_resistor(resistorValue, apply_V, ambient_T,nominal_P, nominal_V))
print("Resistor current is " + str(round(apply_V/resistorValue,3)) + "A")


True
Resistor current is 0.017A


# Jumpers
* 0ohm resistors are maximum 50mOhm.  0603 is rated at 1A.  1206 and 2512 are rated at 2A.

## Use in series with power supplies
* Inductance and resistance must be minimized.  Don't put a ferrite bead in series with a power supply!
* Use the "net-tie in BoM" technique
* Create a PCB variant where the net-tie is not placed.  So boards will be produced with the net-tie, but you can cut it open if you need to (i.e. during testing).  And the full BoM will tell you what components you need to replace it with.
* Create a shorted footprint for the net-tie: using 0.2mm trace (2mOhm when 1mm long at 35µm thickness)
  * ≤1A : 0603 (or 0402)
  * ≤2A : 1206 (or 0805)
  * ≤6A : 1218 (which is a rare size)
* Current sense resistors have a higher current rating, but for 10mOhm or smaller values, you end up with a 2512 package, which is quite big.

In [5]:
# Find the two or three best resistor values to make a resistor divider, based on the resistor set

desired_ratio = 0.523 # Vtop/Vbottom
difference = 1
best_r1 = 0
best_r2 = 0
best_config = 0

# Custom resistor set, according to your BoM
# resistor_set = {10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100} # ohms

# E24 resistor set
resistor_set = {10, 11, 12, 13, 15, 16, 18, 20, 22, 24, 27, 30,
                33, 36, 39, 43, 47, 51, 56, 62, 68, 75, 82, 91,
                100, 110, 120, 130, 150, 160, 180, 200, 220, 240, 270, 300,
                330, 360, 390, 430, 470, 510, 560, 620, 680, 750, 820, 910,
                1000, 1100, 1200, 1300, 1500, 1600, 1800, 2000, 2200, 2400, 2700, 3000,
                3300, 3600, 3900, 4300, 4700, 5100, 5600, 6200, 6800, 7500, 8200, 9100, 10e6} # ohms (10e6 added to simulate setups with only two resistors)

for r1 in [1e6]: # simulate setups with only two resistors.  Only config 3 will lead to a solution.
#for r1 in resistor_set: # simulate setups with three resistors

    for r2 in resistor_set:
        for r3 in resistor_set:
            # Config 1 : R1 / (R2 + R3)
            ratio = r1/(r2+r3)
            if abs(ratio-desired_ratio) < difference:
                difference = abs(ratio-desired_ratio)
                best_r1 = r1
                best_r2 = r2
                best_r3 = r3
                best_config = 1
            # Config 2 : (R1 + R2) / R3
            ratio = (r1+r2)/r3
            if abs(ratio-desired_ratio) < difference:
                difference = abs(ratio-desired_ratio)
                best_r1 = r1
                best_r2 = r2
                best_r3 = r3
                best_config = 2
            # Config 3 : (R1 || R2) / R3
            ratio = ((r1*r2)/(r1+r2))/r3
            if abs(ratio-desired_ratio) < difference:
                difference = abs(ratio-desired_ratio)
                best_r1 = r1
                best_r2 = r2
                best_r3 = r3
                best_config = 3
            # Config 4 : R1 / (R2 || R3)
            ratio = r1/((r2*r3)/(r2+r3))
            if abs(ratio-desired_ratio) < difference:
                difference = abs(ratio-desired_ratio)
                best_r1 = r1
                best_r2 = r2
                best_r3 = r3
                best_config = 4        
            if difference == 0:
                break
        if difference == 0:
            break
    if difference == 0:
        break

error = difference/desired_ratio
print("Best config is " + str(best_config) + " with error of " + str(round(error*100,1)) + "%")
print("R1 = " + str(best_r1) + " ohms")
print("R2 = " + str(best_r2) + " ohms")
print("R3 = " + str(best_r3) + " ohms")

Best config is 3 with error of 0.0%
R1 = 1000000.0 ohms
R2 = 68 ohms
R3 = 130 ohms
